<a href="https://colab.research.google.com/github/Prakash-95s/OpenEduCat/blob/main/GPTTransformersforCustomerservice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.

In [2]:
pip install torch torchvision torchaudio


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
pip install --upgrade transformers

In [4]:
pip install --upgrade transformers torch==2.3.0


In [ ]:
pip install accelerate==0.31.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
pip show transformers torch accelerate

In [ ]:
import pandas as pd

df = pd.read_csv("hf://datasets/Kaludi/Customer-Support-Responses/Customer-Support.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
#Lets provide prompt by combining both query and response
# Combine query and response for training
df['prompt'] = "Query: " + df['query'] + " Response: " + df['response']
df.head()

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from datasets import Dataset
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
import numpy as np

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

dataset = Dataset.from_dict(df)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['prompt'], padding='max_length', truncation=True, return_tensors="pt")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split the dataset
train_dataset, eval_dataset = tokenized_datasets.train_test_split(test_size=0.1).values()

def create_data_loader(dataset, batch_size):
    # Extract input_ids and attention_mask from dataset
    input_ids = [x['input_ids'] for x in dataset]
    attention_mask = [x['attention_mask'] for x in dataset]

    # Convert lists to numpy arrays
    input_ids = np.array(input_ids)
    attention_mask = np.array(attention_mask)

    # Convert numpy arrays to torch tensors
    input_ids_tensor = torch.tensor(input_ids)
    attention_mask_tensor = torch.tensor(attention_mask)

    # Create TensorDataset
    data = TensorDataset(input_ids_tensor, attention_mask_tensor)

    # Choose sampler based on dataset type (train or eval)
    sampler = RandomSampler(data) if dataset == train_dataset else SequentialSampler(data)

    # Create DataLoader
    return DataLoader(data, sampler=sampler, batch_size=batch_size)

# Usage example
train_loader = create_data_loader(train_dataset, batch_size=8)
eval_loader = create_data_loader(eval_dataset, batch_size=8)
# Example function to train for one epoch
def train_epoch(model, loader, optimizer):
    model.train()
    total_loss = 0.0
    for batch in loader:
        input_ids, attention_mask = batch
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), input_ids.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)  # Example optimizer

# Example function to evaluate for one epoch
def eval_epoch(model, loader):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch in loader:
            input_ids, attention_mask = batch
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), input_ids.view(-1))
            total_loss += loss.item()
    return total_loss / len(loader)

# Example training loop
num_epochs = 3
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer)
    eval_loss = eval_epoch(model, eval_loader)
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss}, Eval Loss: {eval_loss}")
# Example prediction
model.eval()
prompt = "I want to return a product."
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(device)
output = model.generate(input_ids, max_length=50, num_return_sequences=1)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


In [ ]:
# Function to generate response
def generate_response(query):
    outputs = model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test the function with a new query
query = "I want to update my payment method."
print(generate_response(query))